In [1]:
import numpy as np
import sklearn
import kmapper as km
import pygeodesic
import pygeodesic.geodesic as geodesic
from scipy.spatial import distance_matrix
import vtk
from mpl_toolkits.mplot3d import Axes3D  # noqa
import scipy as sp
import networkx as nx
import random
import math
from networkx.algorithms import approximation
from scipy.spatial.distance import cdist
import open3d as o3d
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import json

In [2]:
def read_off_file(filename):
    with open(filename) as file:
        lines = file.readlines()
        lines = [line.strip() for line in lines]
        
        if lines[0] != "OFF":
            raise ValueError('Not a valid OFF file')
        n_vertices, n_faces, _ = map(int, lines[1].split())
        vertices = [tuple(map(float, line.split())) for line in lines[2:2 + n_vertices]]
        faces = [tuple(map(int, line.split()[1:])) for line in lines[2 + n_vertices: 2 + n_vertices + n_faces]]

        # Create an array of vertex indices
        vertex_indices = np.arange(n_vertices)
        
    return np.array(vertices), np.array(faces), vertex_indices

vertices_1, faces_1, vertex_indices_1 = read_off_file('octopus1.off')

In [3]:
geoalg = geodesic.PyGeodesicAlgorithmExact(vertices_1, faces_1)
D_1 = np.zeros((len(vertex_indices_1), len(vertex_indices_1)))
for i in range(D_1.shape[0]):
    source_indices = np.array([vertex_indices_1[i]]) 
    target_indices = np.array(vertex_indices_1)
    distancess, best_source = geoalg.geodesicDistances(source_indices, target_indices)
    D_1[i] = distancess

In [4]:
def mapper_subsample(vertices, D, n_add = 10):

    data = vertices
    mapper = km.KeplerMapper(verbose=0)
    
    lens = mapper.fit_transform(data)
    
    graph = mapper.map(
        lens,
        data,
        clusterer=sklearn.cluster.DBSCAN(eps=0.2, min_samples=3),
        cover=km.Cover(n_cubes=60, perc_overlap=0.01),
    )

    nodes = list(graph['nodes'].keys())
    
    indexes = []
    for i in range(len(nodes)):
        tmp_idx = np.array(graph['nodes'][nodes[i]], int)
        tmp_d = D[tmp_idx,:][:, tmp_idx].sum(axis=1)
        indexes.append(tmp_idx[np.argmin(tmp_d)])

    for i in range(n_add):
        tmp_D = np.min(D[:,indexes], axis=1)
        indexes.append(np.argmax(tmp_D))

    print("%d points selected by Mapper with %d addition points" %(len(nodes), n_add))

    return indexes

In [5]:
idx_1 = mapper_subsample(vertices_1, D_1, n_add=10)

134 points selected by Mapper with 10 addition points


/Users/salovjade/anaconda3/lib/python3.10/site-packages/threadpoolctl.py:1010: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [6]:
highlightIndices1 = np.array(idx_1)
highlightIndices1

array([ 307,  556,  575,   53,  148,  414,   57,  956,   55,  339,  628,
        971,  574,  909,    1,  602,  463, 1001,    0,   69,  358,  992,
        218,   96,  906,  999,  582,  623,  914,  687,  621,  800,  354,
        994,  831,  836,  616,  993,  352,  686,  834,  349,   81,  742,
        997,  752,  608,   83,  547,  784,  987,  667,  362,  392,  249,
        809,  984,  400,  629,  265,  102,  883,  716,  258,  355,  678,
        160,  100,  581,  782,  931,  254,  459,  775,  882,  618,  756,
        939,  159,  351,  700,  761,    7,  693,  759,  748,  416,  731,
        268,  270,  136,  708,  818,  174,   17,  523,  263,  721,  703,
        106,  634,  288,  114,  255,  165,  509,  198,   38,  367,  712,
        276,   28,  105,  550,  475,  266,  727,  157,  172,  298,  156,
        273,  668,  389,  421,  378,  747,  181,  204,  112,  554,  501,
         33,  173,  935,  898,  948,   64,  597,  968,  947,  944,  328,
        539])

In [7]:
#leg1

In [8]:
geoalg = geodesic.PyGeodesicAlgorithmExact(vertices_1, faces_1)

test, best = geoalg.geodesicDistances([550-1], [832-1])

source_indices = np.array([550-1]) 
target_indices = np.array(vertex_indices_1)
distancess, best_source = geoalg.geodesicDistances(source_indices, target_indices)

target_indices = np.array(vertex_indices_1)

indices_less_than_test_1 = target_indices[distancess < test]
leg1 = indices_less_than_test_1.tolist()
indices_less_than_test_1

array([  0,   1,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58, 123, 124, 125, 126, 127, 128, 129, 130, 131, 135,
       196, 203, 206, 207, 209, 210, 211, 214, 215, 216, 218, 219, 220,
       294, 299, 301, 302, 305, 306, 307, 309, 311, 312, 314, 316, 317,
       318, 319, 407, 411, 413, 414, 417, 418, 419, 420, 422, 426, 427,
       430, 431, 432, 436, 548, 549, 556, 560, 562, 563, 564, 565, 567,
       568, 571, 572, 573, 574, 575, 578, 579, 582, 671, 681, 687, 689,
       690, 745, 746, 749, 753, 794, 797, 800, 828])

In [9]:
#leg2

In [10]:
geoalg = geodesic.PyGeodesicAlgorithmExact(vertices_1, faces_1)

test, best = geoalg.geodesicDistances([456-1], [841-1])

source_indices = np.array([456-1]) 
target_indices = np.array(vertex_indices_1)
distancess, best_source = geoalg.geodesicDistances(source_indices, target_indices)

target_indices = np.array(vertex_indices_1)

indices_less_than_test_2 = target_indices[distancess < test]
leg2 = indices_less_than_test_2.tolist()
indices_less_than_test_2

array([  2,  64,  65,  67,  69,  71,  72,  75,  76,  77,  78,  82, 137,
       138, 140, 142, 146, 147, 148, 223, 227, 228, 230, 231, 232, 239,
       240, 324, 325, 329, 331, 335, 336, 338, 339, 340, 341, 343, 345,
       424, 434, 435, 440, 442, 445, 446, 451, 454, 455, 584, 585, 589,
       590, 593, 596, 598, 599, 602, 603, 604, 605, 610, 611, 688, 765,
       769, 771, 806, 834])

In [11]:
#leg3

In [12]:
geoalg = geodesic.PyGeodesicAlgorithmExact(vertices_1, faces_1)

test, best = geoalg.geodesicDistances([248-1], [842-1])

source_indices = np.array([248-1]) 
target_indices = np.array(vertex_indices_1)
distancess, best_source = geoalg.geodesicDistances(source_indices, target_indices)

target_indices = np.array(vertex_indices_1)

indices_less_than_test_3 = target_indices[distancess < test]
leg3 = indices_less_than_test_3.tolist()
indices_less_than_test_3

array([ 63,  73,  74,  80,  81,  83,  86,  87,  89,  92,  93,  94,  96,
        98, 141, 144, 149, 151, 153, 221, 226, 237, 238, 241, 243, 245,
       246, 247, 248, 321, 322, 328, 330, 333, 337, 344, 348, 349, 352,
       353, 354, 356, 358, 438, 441, 448, 449, 453, 457, 460, 461, 462,
       463, 464, 465, 466, 467, 470, 583, 600, 608, 609, 615, 616, 620,
       621, 623, 625, 627, 628, 694, 695, 696, 697, 698, 770, 808, 809])

In [13]:
#leg4

In [14]:
geoalg = geodesic.PyGeodesicAlgorithmExact(vertices_1, faces_1)

test, best = geoalg.geodesicDistances([250-1], [799-1])

source_indices = np.array([250-1]) 
target_indices = np.array(vertex_indices_1)
distancess, best_source = geoalg.geodesicDistances(source_indices, target_indices)

target_indices = np.array(vertex_indices_1)

indices_less_than_test_4 = target_indices[distancess < test]
leg4 = indices_less_than_test_4.tolist()
indices_less_than_test_4

array([ 59,  60,  61,  62,  66,  68,  70,  79,  84,  85,  88,  90,  91,
        95,  97,  99, 100, 101, 102, 132, 133, 134, 136, 139, 143, 145,
       150, 152, 154, 213, 222, 224, 225, 229, 233, 234, 235, 236, 242,
       244, 249, 320, 323, 326, 327, 332, 334, 342, 346, 347, 350, 351,
       355, 357, 359, 360, 361, 362, 416, 428, 437, 439, 443, 444, 447,
       450, 452, 456, 458, 459, 468, 469, 576, 586, 587, 588, 591, 592,
       594, 595, 597, 601, 606, 607, 612, 613, 614, 617, 618, 619, 622,
       624, 626, 629, 630, 631, 632, 683, 684, 692, 693, 755, 759, 760,
       761, 795, 796])

In [15]:
#leg5

In [16]:
geoalg = geodesic.PyGeodesicAlgorithmExact(vertices_1, faces_1)

test, best = geoalg.geodesicDistances([434-1], [819-1])

source_indices = np.array([434-1]) 
target_indices = np.array(vertex_indices_1)
distancess, best_source = geoalg.geodesicDistances(source_indices, target_indices)

target_indices = np.array(vertex_indices_1)

indices_less_than_test_5 = target_indices[distancess < test]
leg5 = indices_less_than_test_5.tolist()
indices_less_than_test_5

array([194, 195, 197, 198, 199, 200, 201, 202, 204, 205, 208, 212, 217,
       287, 288, 293, 295, 296, 297, 298, 300, 303, 304, 308, 313, 402,
       408, 409, 410, 412, 415, 421, 423, 433, 522, 525, 526, 527, 531,
       532, 542, 544, 550, 551, 552, 554, 555, 561, 642, 643, 647, 655,
       656, 657, 658, 660, 662, 663, 665, 668, 677, 682, 691, 703, 706,
       707, 712, 718, 719, 720, 721, 723, 724, 726, 727, 728, 729, 730,
       733, 734, 735, 740, 743, 747, 750, 754, 758, 766, 767, 780, 787,
       789])

In [17]:
#leg6

In [18]:
geoalg = geodesic.PyGeodesicAlgorithmExact(vertices_1, faces_1)

test, best = geoalg.geodesicDistances([276-1], [775-1])

source_indices = np.array([276-1]) 
target_indices = np.array(vertex_indices_1)
distancess, best_source = geoalg.geodesicDistances(source_indices, target_indices)

target_indices = np.array(vertex_indices_1)

indices_less_than_test_6 = target_indices[distancess < test]
leg6 = indices_less_than_test_6.tolist()
indices_less_than_test_6

array([ 22,  23,  24,  26,  27,  28,  30,  31,  32,  33,  37,  38,  39,
        41, 108, 110, 111, 112, 113, 114, 118, 119, 121, 170, 171, 172,
       173, 176, 178, 179, 180, 181, 182, 186, 188, 189, 262, 266, 267,
       273, 275, 276, 277, 278, 279, 284, 377, 378, 381, 383, 384, 388,
       389, 390, 391, 393, 396, 404, 491, 492, 493, 494, 495, 498, 499,
       501, 502, 503, 504, 508, 509, 510, 512, 521, 523, 528, 634, 635,
       636, 644, 646, 699, 702, 772, 778])

In [19]:
#leg7

In [20]:
geoalg = geodesic.PyGeodesicAlgorithmExact(vertices_1, faces_1)

test, best = geoalg.geodesicDistances([104-1], [818-1])

source_indices = np.array([104-1]) 
target_indices = np.array(vertex_indices_1)
distancess, best_source = geoalg.geodesicDistances(source_indices, target_indices)

target_indices = np.array(vertex_indices_1)

indices_less_than_test_7 = target_indices[distancess < test]
leg7 = indices_less_than_test_7.tolist()
indices_less_than_test_7

array([  5,   8,   9,  10,  14,  15,  16,  17,  25,  29,  36, 103, 105,
       106, 109, 115, 117, 156, 157, 158, 161, 162, 163, 165, 174, 175,
       177, 183, 187, 251, 252, 253, 255, 257, 260, 261, 263, 264, 268,
       270, 280, 281, 283, 365, 366, 367, 370, 372, 374, 379, 385, 386,
       399, 405, 475, 476, 479, 480, 481, 483, 484, 485, 488, 496, 497,
       505, 506, 514, 515, 518, 524, 537, 633, 639, 700, 701, 711, 773,
       775, 776, 783])

In [21]:
#leg8

In [22]:
geoalg = geodesic.PyGeodesicAlgorithmExact(vertices_1, faces_1)

test, best = geoalg.geodesicDistances([4-1], [845-1])

source_indices = np.array([4-1]) 
target_indices = np.array(vertex_indices_1)
distancess, best_source = geoalg.geodesicDistances(source_indices, target_indices)

target_indices = np.array(vertex_indices_1)

indices_less_than_test_8 = target_indices[distancess < test]
leg8 = indices_less_than_test_8.tolist()
indices_less_than_test_8

array([  3,   4,   6,   7,  11,  12,  13,  18,  19,  20,  21,  34,  35,
        40,  42,  43,  44, 104, 107, 116, 120, 122, 155, 159, 160, 164,
       166, 167, 168, 169, 184, 185, 190, 191, 192, 193, 250, 254, 256,
       258, 259, 265, 269, 271, 272, 274, 282, 286, 289, 290, 292, 363,
       364, 368, 369, 371, 373, 375, 376, 380, 382, 387, 392, 394, 395,
       397, 400, 401, 406, 471, 472, 473, 474, 477, 478, 482, 486, 487,
       489, 490, 500, 507, 511, 513, 516, 517, 519, 520, 529, 530, 640,
       641, 649, 704, 705, 717, 737, 785, 788, 821, 822])

In [23]:
#body

In [25]:
combined = leg1 + leg2 + leg3 + leg4 + leg5 + leg6 + leg7 + leg8
body = [item for item in vertex_indices_1 if item not in combined]

indices_less_than_test_9 = np.array(body)
indices_less_than_test_9

array([ 285,  291,  310,  315,  398,  403,  425,  429,  533,  534,  535,
        536,  538,  539,  540,  541,  543,  545,  546,  547,  553,  557,
        558,  559,  566,  569,  570,  577,  580,  581,  637,  638,  645,
        648,  650,  651,  652,  653,  654,  659,  661,  664,  666,  667,
        669,  670,  672,  673,  674,  675,  676,  678,  679,  680,  685,
        686,  708,  709,  710,  713,  714,  715,  716,  722,  725,  731,
        732,  736,  738,  739,  741,  742,  744,  748,  751,  752,  756,
        757,  762,  763,  764,  768,  774,  777,  779,  781,  782,  784,
        786,  790,  791,  792,  793,  798,  799,  801,  802,  803,  804,
        805,  807,  810,  811,  812,  813,  814,  815,  816,  817,  818,
        819,  820,  823,  824,  825,  826,  827,  829,  830,  831,  832,
        833,  835,  836,  837,  838,  839,  840,  841,  842,  843,  844,
        845,  846,  847,  848,  849,  850,  851,  852,  853,  854,  855,
        856,  857,  858,  859,  860,  861,  862,  8

In [26]:
#import idx_2

In [27]:
vertices_2, faces_2, vertex_indices_2 = read_off_file('octopus2.off')

geoalg = geodesic.PyGeodesicAlgorithmExact(vertices_2, faces_2)
D_2 = np.zeros((len(vertex_indices_2), len(vertex_indices_2)))
for i in range(D_2.shape[0]):
    source_indices = np.array([vertex_indices_2[i]]) 
    target_indices = np.array(vertex_indices_2)
    distancess, best_source = geoalg.geodesicDistances(source_indices, target_indices)
    D_1[i] = distancess
    
idx_2 = mapper_subsample(vertices_2, D_2, n_add=10)  

highlightIndices2 = np.array(idx_2)
highlightIndices2

117 points selected by Mapper with 10 addition points


array([925, 841, 843, 817, 818, 776, 773, 753, 684, 682, 983, 614, 957,
       579, 448, 934, 208, 510, 874, 169, 421, 587, 889, 106, 419, 835,
       105, 852, 882, 105, 794, 125, 770, 881, 124, 418, 699, 168, 480,
       416, 976, 424, 356, 314, 381, 955, 230, 404, 182, 790, 812, 166,
       379, 164, 162, 458, 853, 177, 437, 250, 375, 655, 830, 244, 238,
       242, 805, 868, 192, 378, 865, 189, 220, 821, 133, 240, 849, 115,
       139, 333, 810, 119, 272,  56,  99, 436,  63,  78, 200, 468,  55,
        71, 273, 493,  61, 159, 465,  60, 143, 492,  59, 123, 528,  25,
        39,   8,  30,   1,   0,  20,   6,  21,  16,  12,  11,   2,   4,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [28]:
#assign legs matching from each P

In [37]:
thresholds = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]



def find_connected_indices(subset_highlightIndices1, matrix, highlightIndices1, highlightIndices2):
    connected_indices_subset = []
    for i, index1 in enumerate(highlightIndices1):
        if index1 in subset_highlightIndices1:
            # Find the index of the max value in the row
            j = matrix.iloc[i].idxmax()
            
            # Check if the max value is not zero
            if matrix.iat[i, j] != 0:
                connected_indices_subset.append(highlightIndices2[j])
    return np.array(connected_indices_subset)


for threshold in thresholds:    
    file_path = f'P_octopus_1203_{threshold}.csv'
    matrix_df = pd.read_csv(file_path, header=None)
    matrix_sum = np.sum(matrix_df)

    print("Threshold:", threshold, "Sum of the matrix:", matrix_sum)
    
    # Define the index arrays as provided by the user
    highlightIndices1 = idx_1 
    highlightIndices2 = idx_2

    ## Find non-zero entries in the matrix and map them to the corresponding indices in highlightIndices1 and highlightIndices2
    #connected_indices = []
    #for i in range(matrix_df.shape[0]):  # Loop over rows
       # for j in range(matrix_df.shape[1]):  # Loop over columns
          #  if matrix_df.iat[i, j] != 0:  # Check if the matrix element is non-zero
           #     connected_indices.append((highlightIndices1[i], highlightIndices2[j]))  # Store the connected indices

    # Find the column with the largest value in each row and map them
    connected_indices = []
    for i in range(matrix_df.shape[0]):  # Loop over rows
        # Find the index of the max value in the row
        j = matrix_df.iloc[i].idxmax()

        # Map and store the indices if the max value is not zero
        if matrix_df.iat[i, j] != 0:
            connected_indices.append((highlightIndices1[i], highlightIndices2[j]))

    ## Find the row with the largest value in each column and map them
    #connected_indices = []
    #for j in range(matrix_df.shape[1]):  # Loop over columns
       # # Find the index of the max value in the column
       # i = matrix_df.iloc[:, j].idxmax()

       # # Map and store the indices if the max value is not zero
       # if matrix_df.iat[i, j] != 0:
          #  connected_indices.append((highlightIndices1[i], highlightIndices2[j]))

    # Display the connected indices
    # connected_indices
    
    # Convert connected_indices to a DataFrame
    connected_indices_df = pd.DataFrame(connected_indices, columns=['Index1', 'Index2'])

    first_column = connected_indices_df['Index1']

    # Optionally, convert it to a numpy array
    sub_highlightIndices1 = first_column.to_numpy()
    
    second_column = connected_indices_df['Index2']

    # Optionally, convert it to a numpy array
    sub_highlightIndices2 = second_column.to_numpy()
    
    # Example usage:
    sub_highlightIndices1_leg1 = np.intersect1d(sub_highlightIndices1,indices_less_than_test_1)
    # Read from matlab first, remember -1 
    
    sub_highlightIndices2_leg1 = find_connected_indices(sub_highlightIndices1_leg1, matrix_df, highlightIndices1, highlightIndices2)
    
    sub_highlightIndices1_leg2 = np.intersect1d(sub_highlightIndices1,indices_less_than_test_2)

    # Read from matlab first, remember -1 
    sub_highlightIndices2_leg2 = find_connected_indices(sub_highlightIndices1_leg2, matrix_df, highlightIndices1, highlightIndices2)

    
    sub_highlightIndices1_leg3 = np.intersect1d(sub_highlightIndices1,indices_less_than_test_3)

    # Read from matlab first, remember -1 
    sub_highlightIndices2_leg3 = find_connected_indices(sub_highlightIndices1_leg3, matrix_df, highlightIndices1, highlightIndices2)

    
    sub_highlightIndices1_leg4 = np.intersect1d(sub_highlightIndices1,indices_less_than_test_4)

    # Read from matlab first, remember -1 
    sub_highlightIndices2_leg4 = find_connected_indices(sub_highlightIndices1_leg4, matrix_df, highlightIndices1, highlightIndices2)

    
    sub_highlightIndices1_leg5 = np.intersect1d(sub_highlightIndices1,indices_less_than_test_5)

    # Read from matlab first, remember -1 
    sub_highlightIndices2_leg5 = find_connected_indices(sub_highlightIndices1_leg5, matrix_df, highlightIndices1, highlightIndices2)

    
    sub_highlightIndices1_leg6 = np.intersect1d(sub_highlightIndices1,indices_less_than_test_6)

    # Read from matlab first, remember -1 
    sub_highlightIndices2_leg6 = find_connected_indices(sub_highlightIndices1_leg6, matrix_df, highlightIndices1, highlightIndices2)

    
    sub_highlightIndices1_leg7 = np.intersect1d(sub_highlightIndices1,indices_less_than_test_7)

    # Read from matlab first, remember -1 
    sub_highlightIndices2_leg7 = find_connected_indices(sub_highlightIndices1_leg7, matrix_df, highlightIndices1, highlightIndices2)

    
    sub_highlightIndices1_leg8 = np.intersect1d(sub_highlightIndices1,indices_less_than_test_8)

    # Read from matlab first, remember -1 
    sub_highlightIndices2_leg8 = find_connected_indices(sub_highlightIndices1_leg8, matrix_df, highlightIndices1, highlightIndices2)

    
    sub_highlightIndices1_body = np.intersect1d(sub_highlightIndices1,indices_less_than_test_9)

    # Read from matlab first, remember -1 
    sub_highlightIndices2_body = find_connected_indices(sub_highlightIndices1_body, matrix_df, highlightIndices1, highlightIndices2)

    
    # save data
    data_to_save = {
    "sub_highlightIndices1": sub_highlightIndices1.tolist(),
    "sub_highlightIndices2": sub_highlightIndices2.tolist(), 
    "sub_highlightIndices1_body": sub_highlightIndices1_body.tolist(),
    "sub_highlightIndices2_body": sub_highlightIndices2_body.tolist(),
    "sub_highlightIndices1_leg1": sub_highlightIndices1_leg1.tolist(),
    "sub_highlightIndices2_leg1": sub_highlightIndices2_leg1.tolist(),
    "sub_highlightIndices1_leg2": sub_highlightIndices1_leg2.tolist(),
    "sub_highlightIndices2_leg2": sub_highlightIndices2_leg2.tolist(),
    "sub_highlightIndices1_leg3": sub_highlightIndices1_leg3.tolist(),
    "sub_highlightIndices2_leg3": sub_highlightIndices2_leg3.tolist(),
    "sub_highlightIndices1_leg4": sub_highlightIndices1_leg4.tolist(),
    "sub_highlightIndices2_leg4": sub_highlightIndices2_leg4.tolist(),
    "sub_highlightIndices1_leg5": sub_highlightIndices1_leg5.tolist(),
    "sub_highlightIndices2_leg5": sub_highlightIndices2_leg5.tolist(),
    "sub_highlightIndices1_leg6": sub_highlightIndices1_leg6.tolist(),
    "sub_highlightIndices2_leg6": sub_highlightIndices2_leg6.tolist(),
    "sub_highlightIndices1_leg7": sub_highlightIndices1_leg7.tolist(),
    "sub_highlightIndices2_leg7": sub_highlightIndices2_leg7.tolist(),
    "sub_highlightIndices1_leg8": sub_highlightIndices1_leg8.tolist(),
    "sub_highlightIndices2_leg8": sub_highlightIndices2_leg8.tolist(),
    }

    # Save as JSON file
    filename = f'highlight_indices_{threshold}.json'
    with open(filename, 'w') as outfile:
        json.dump(data_to_save, outfile)

Threshold: 0.05 Sum of the matrix: 0      0.00000
1      0.00000
2      0.00000
3      0.00000
4      0.00000
        ...   
122    0.00121
123    0.00121
124    0.00121
125    0.00121
126    0.00121
Length: 127, dtype: float64
Threshold: 0.1 Sum of the matrix: 0      0.000000
1      0.000000
2      0.000000
3      0.007874
4      0.006957
         ...   
122    0.000000
123    0.000000
124    0.000000
125    0.000000
126    0.000000
Length: 127, dtype: float64
Threshold: 0.2 Sum of the matrix: 0      0.000000
1      0.000000
2      0.002318
3      0.004689
4      0.007874
         ...   
122    0.000000
123    0.000000
124    0.000000
125    0.000000
126    0.000000
Length: 127, dtype: float64
Threshold: 0.3 Sum of the matrix: 0      0.000000
1      0.000000
2      0.001590
3      0.003598
4      0.007330
         ...   
122    0.000000
123    0.000000
124    0.000000
125    0.000000
126    0.000000
Length: 127, dtype: float64
Threshold: 0.4 Sum of the matrix: 0      0.000000
1      0

In [ ]:
#test sum P

In [39]:
for threshold in thresholds:
    filename = f'P_octopus_1203_{threshold}.csv'
    df = pd.read_csv(filename)
    matrix = df.values
    matrix_sum = np.sum(matrix)
    print("rho:", threshold, "Sum of the matrix:", matrix_sum)
    

rho: 0.05 Sum of the matrix: 0.1259295708
rho: 0.1 Sum of the matrix: 0.3435586171
rho: 0.2 Sum of the matrix: 0.4913057736
rho: 0.3 Sum of the matrix: 0.5918088347999999
rho: 0.4 Sum of the matrix: 0.6765638666000001
rho: 0.5 Sum of the matrix: 0.6992563436
rho: 0.6 Sum of the matrix: 0.7603893277
rho: 0.7 Sum of the matrix: 0.7507108502000001
rho: 0.8 Sum of the matrix: 0.6859142612
rho: 0.9 Sum of the matrix: 0.7048884516999999
rho: 1.0 Sum of the matrix: 0.68591426
rho: 1.1 Sum of the matrix: 0.7271434824999999
rho: 1.2 Sum of the matrix: 0.7123250195
rho: 1.3 Sum of the matrix: 0.7696337055
rho: 1.4 Sum of the matrix: 0.8114913248
rho: 1.5 Sum of the matrix: 0.8319506028
rho: 1.6 Sum of the matrix: 0.8211395309
rho: 1.7 Sum of the matrix: 0.8670166097
rho: 1.8 Sum of the matrix: 0.8832567689999999
rho: 1.9 Sum of the matrix: 0.956911624
rho: 2.0 Sum of the matrix: 0.9842519446999999
